In [1]:
import os
from typing import Callable

###
### The code in this block isabout loading results
###

def load_single_rustls_output(path: str) -> [(str, float)]:
    measurements = []
    with open(path) as file:
        for line in file:
            if line.startswith('target'):
                continue

            parts = line.split()
            name = "_".join(parts[0:-2])
            measurement = float(parts[-2])

            measurements.append((name, measurement))

    # The ChaCha20 benchmark runs twice, remove the second run
    cha_cha_indexes = [i for i in range(0, len(measurements)) if 'TLS_ECDHE_RSA_WITH_CHACHA20_POLY1305_SHA256' in measurements[i][0]]
    if len(cha_cha_indexes) == 4:
        del measurements[cha_cha_indexes[2]]
        del measurements[cha_cha_indexes[2]]

    return measurements


def load_single_openssl_output(path: str) -> list[(str, float)]:
    measurements = []
    with open(path) as file:
        lines = [line for line in file]
        i = 0
        while i < len(lines):
            line = lines[i]
            if line.startswith('env'):
                command_line = lines[i]
                command_start = command_line.find('./bench')
                command_parts = command_line[command_start:].split()

                tls13 = 'TLS' in command_parts[2]
                command_parts[2] = command_parts[2].replace('TLS_', '')

                command_parts.insert(2, 'TLS13' if tls13 else 'TLS12')

                command = "_".join(command_parts[1:])

                if command_parts[1] == 'bulk':
                    send_line = lines[i + 2]
                    recv_line = lines[i + 3]

                    sent = float(send_line.split()[1])
                    received = float(recv_line.split()[1])

                    measurements.append((f'{command}_sent', sent))
                    measurements.append((f'{command}_received', received))

                    i += 4
                    continue
                elif command_parts[1] == 'handshake':
                    send_line = lines[i + 1]
                    recv_line = lines[i + 2]

                    sent = float(send_line.split()[2])
                    received = float(recv_line.split()[2])

                    measurements.append((f'{command}_client', sent))
                    measurements.append((f'{command}_server', received))

                    i += 3
                    continue
                elif command_parts[1] == 'handshake-resume' or command_parts[1] == 'handshake-ticket':
                    send_line = lines[i + 4]
                    recv_line = lines[i + 5]

                    sent = float(send_line.split()[2])
                    received = float(recv_line.split()[2])

                    measurements.append((f'{command}_client', sent))
                    measurements.append((f'{command}_server', received))

                    i += 6
                    continue

            i += 1

    return measurements


def load_outputs(dir: str, loader: Callable[[str], list[(str, float)]]) -> list[list[(str, float)]]:
    return [loader(os.path.join(dir, path)) for path in os.listdir(dir)]


class OpenSSL:
    def __init__(self, version: str):
        self.version = version

    def load_outputs(self) -> list[list[(str, float)]]:
        return load_outputs(f'data/openssl-throughput/openssl_{self.version}', load_single_openssl_output)

    def column_name(self) -> str:
        return f'OpenSSL ({self.version})'


class Rustls:
    def __init__(self, version: str, crypto_backend: str, allocator: str):
        self.version = version
        self.crypto_backend = crypto_backend
        self.allocator = allocator

    def load_outputs(self) -> list[list[(str, float)]]:
        return load_outputs(f'data/rustls-throughput/rustls_{self.version}_{self.crypto_backend}_{self.allocator}', load_single_rustls_output)

    def column_name(self) -> str:
        return f'Rustls ({self.version}, {self.crypto_backend}, {self.allocator})'

# Test that loading works properly
Rustls('0.22.0', 'ring_gcc', 'malloc').load_outputs()
OpenSSL('3.2.0_clang').load_outputs()

# The output for each run always follows the same order. Each item in the list below corresponds to
# the benchmark results in that position.
scenarios = [
    # bulk TLS12_ECDHE-RSA-AES128-GCM-SHA128
    ('bulk', '1.2', 'ECDHE', 'RSA', 'AES128-GCM', 'SHA256', 'sent'),
    ('bulk', '1.2', 'ECDHE', 'RSA', 'AES128-GCM', 'SHA256', 'received'),
    # bulk TLS12_ECDHE-RSA-AES256-GCM-SHA384
    ('bulk', '1.2', 'ECDHE', 'RSA', 'AES256-GCM', 'SHA384', 'sent'),
    ('bulk', '1.2', 'ECDHE', 'RSA', 'AES256-GCM', 'SHA384', 'received'),
    # bulk TLS12_ECDHE-RSA-CHACHA20-POLY1305
    ('bulk', '1.2', 'ECDHE', 'RSA', 'CHACHA20', 'POLY1305', 'sent'),
    ('bulk', '1.2', 'ECDHE', 'RSA', 'CHACHA20', 'POLY1305', 'received'),
    # bulk TLS13_AES_256_GCM_SHA384
    ('bulk', '1.3', 'ECDHE', 'RSA', 'AES256-GCM', 'SHA384', 'sent'),
    ('bulk', '1.3', 'ECDHE', 'RSA', 'AES256-GCM', 'SHA384', 'received'),
    # handshake TLS12_ECDHE-RSA-AES256-GCM-SHA384
    ('handshake-full', '1.2', 'ECDHE', 'RSA', 'AES256-GCM', 'SHA384', 'client'),
    ('handshake-full', '1.2', 'ECDHE', 'RSA', 'AES256-GCM', 'SHA384', 'server'),
    ('handshake-session-id', '1.2', 'ECDHE', 'RSA', 'AES256-GCM', 'SHA384', 'client'),
    ('handshake-session-id', '1.2', 'ECDHE', 'RSA', 'AES256-GCM', 'SHA384', 'server'),
    ('handshake-ticket', '1.2', 'ECDHE', 'RSA', 'AES256-GCM', 'SHA384', 'client'),
    ('handshake-ticket', '1.2', 'ECDHE', 'RSA', 'AES256-GCM', 'SHA384', 'server'),
    # handshake TLS13_ECDHE-RSA-AES256-GCM-SHA384
    ('handshake-full', '1.3', 'ECDHE', 'RSA', 'AES256-GCM', 'SHA384', 'client'),
    ('handshake-full', '1.3', 'ECDHE', 'RSA', 'AES256-GCM', 'SHA384', 'server'),
    ('handshake-session-id', '1.3', 'ECDHE', 'RSA', 'AES256-GCM', 'SHA384', 'client'),
    ('handshake-session-id', '1.3', 'ECDHE', 'RSA', 'AES256-GCM', 'SHA384', 'server'),
    ('handshake-ticket', '1.3', 'ECDHE', 'RSA', 'AES256-GCM', 'SHA384', 'client'),
    ('handshake-ticket', '1.3', 'ECDHE', 'RSA', 'AES256-GCM', 'SHA384', 'server'),
]

In [6]:
###
### The code in this block is about comparing benchmark runs
###

def results_per_bench(outputs: list[list[(str, float)]]) -> list[list[float]]:
    results = [[] for _ in scenarios]
    for output in outputs:
        i = 0
        for (_, result) in output:
            results[i].append(result)
            i += 1

    return results

def median(values: list[float]) -> float:
    values.sort()
    return values[int(len(values) / 2)]

def table(name1, results1, name2, results2):
    results = [(scenario, median(result1), median(result2)) for (scenario, (result1, result2)) in zip(scenarios, zip(results1, results2))]
    print(f'|Scenario|{name1}|{name2}|Factor|')
    print('|-|-:|-:|-:|')
    for (scenario, r1, r2) in results:
        factor = r2 / r1
        print(f'|{"_".join(scenario)}|{r1}|{r2}|{factor:.2f}x|')

# Other libs:
# - OpenSSL('3.2.0_clang')
# - Rustls('0.22.0', 'aws-lc_gcc', 'malloc')
# - Rustls('0.22.0', 'ring_clang', 'jemalloc')
# - ...

lib1 = OpenSSL('3.2.0_gcc')
lib2 = Rustls('0.22.0', 'aws-lc_gcc', 'jemalloc')
table(lib1.column_name(), results_per_bench(lib1.load_outputs()), lib2.column_name(), results_per_bench(lib2.load_outputs()))

|Scenario|OpenSSL (3.2.0_gcc)|Rustls (0.22.0, aws-lc_gcc, jemalloc)|Factor|
|-|-:|-:|-:|
|bulk_1.2_ECDHE_RSA_AES128-GCM_SHA256_sent|6503.58|6712.29|1.03x|
|bulk_1.2_ECDHE_RSA_AES128-GCM_SHA256_received|7234.11|6017.82|0.83x|
|bulk_1.2_ECDHE_RSA_AES256-GCM_SHA384_sent|6162.57|6193.18|1.00x|
|bulk_1.2_ECDHE_RSA_AES256-GCM_SHA384_received|6609.27|5669.77|0.86x|
|bulk_1.2_ECDHE_RSA_CHACHA20_POLY1305_sent|2998.06|1750.11|0.58x|
|bulk_1.2_ECDHE_RSA_CHACHA20_POLY1305_received|3107.17|1731.27|0.56x|
|bulk_1.3_ECDHE_RSA_AES256-GCM_SHA384_sent|6041.82|6256.94|1.04x|
|bulk_1.3_ECDHE_RSA_AES256-GCM_SHA384_received|6406.44|5945.02|0.93x|
|handshake-full_1.2_ECDHE_RSA_AES256-GCM_SHA384_client|2581.16|4991.73|1.93x|
|handshake-full_1.2_ECDHE_RSA_AES256-GCM_SHA384_server|2120.95|1485.95|0.70x|
|handshake-session-id_1.2_ECDHE_RSA_AES256-GCM_SHA384_client|21514.4|49893.99|2.32x|
|handshake-session-id_1.2_ECDHE_RSA_AES256-GCM_SHA384_server|23183.8|41846.34|1.80x|
|handshake-ticket_1.2_ECDHE_RSA_AES256-GC